## DLT Pipelines

**Imports**

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

**Creating the streaming source**

**Streaming Table**

**Creating the expectations (constraints) before tables**

In [0]:
expectations = {
  "rule1": "product_id IS NOT NULL",
  "rule2": "product_name IS NOT NULL"
}

In [0]:
@dlt.table()

@dlt.expect_all_or_dop(expectations) 
def stage_dim_products():

  df = spark.readStream.table(`catalog-databricks`.silver.products_silver)
  # If we want We can do the transformations here.
  return df 

**Streaming Views**

In [0]:
@dlt.view
def dim_products_v():

  df =spark.readstream.table("Live.stage_dim_products")
  return df

**Dim Products**

In [0]:
dlt.createStreamingTable("dim_products")

In [0]:
dlt.apply_changes(
  target = "dim_products",
  source = "Live.dim_products_v",
  keys = ["product_id"],
  sequence_by = "product_id",
#   ignore_null_updates = False,
#   apply_as_deletes = None,
#   apply_as_truncates = None,
#   column_list = None,
#   except_column_list = None,
  stored_as_scd_type = 2,
#   track_history_column_list = None,
#   track_history_except_column_list = None
)